# Automated Sensing of Red vs. White Wine
_... with the cheapest sensor possible_

#### Data Science 420 BB
#### University of Washington Professional & Continuing Education

#### Homework 1: Low-cost Sensor
#### Leo Salemann, 4/11/18

# LOAD DATA

In [35]:
file <- "./RedWhiteWine.csv"
all_data <- read.csv(file, header=TRUE, sep=",", stringsAsFactors = FALSE)

# If the target column, say column class, is numerical, you need to convert it to factors in R for classification problem
all_data$Class <- factor(all_data$Class)
head(all_data)


fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,Class
7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5,1
7.8,0.88,0.00,2.6,0.098,25,67,0.9968,3.20,0.68,9.8,5,1
7.8,0.76,0.04,2.3,0.092,15,54,0.9970,3.26,0.65,9.8,5,1
11.2,0.28,0.56,1.9,0.075,17,60,0.9980,3.16,0.58,9.8,6,1
7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5,1
7.4,0.66,0.00,1.8,0.075,13,40,0.9978,3.51,0.56,9.4,5,1


## Redefine Class 1 as Red and 0 as White

## Drop the quality attribute

In [36]:
library("dplyr")

In [37]:
all_data2 = select(all_data, -quality)
head(all_data2)

fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,Class
7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,1
7.8,0.88,0.00,2.6,0.098,25,67,0.9968,3.20,0.68,9.8,1
7.8,0.76,0.04,2.3,0.092,15,54,0.9970,3.26,0.65,9.8,1
11.2,0.28,0.56,1.9,0.075,17,60,0.9980,3.16,0.58,9.8,1
7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,1
7.4,0.66,0.00,1.8,0.075,13,40,0.9978,3.51,0.56,9.4,1


## Reduce to the features our sensor technology can acutually support

In [38]:
data = select(all_data, fixed.acidity, volatile.acidity, citric.acid, residual.sugar, chlorides, density, Class)
# data = select(all_data, fixed.acidity, volatile.acidity, citric.acid, residual.sugar, chlorides, density, Class, Color)
head(data)
dim(data)
summary(data)

fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,density,Class
7.4,0.70,0.00,1.9,0.076,0.9978,1
7.8,0.88,0.00,2.6,0.098,0.9968,1
7.8,0.76,0.04,2.3,0.092,0.9970,1
11.2,0.28,0.56,1.9,0.075,0.9980,1
7.4,0.70,0.00,1.9,0.076,0.9978,1
7.4,0.66,0.00,1.8,0.075,0.9978,1


[1] 6497    7

 fixed.acidity    volatile.acidity  citric.acid     residual.sugar  
 Min.   : 3.800   Min.   :0.0800   Min.   :0.0000   Min.   : 0.600  
 1st Qu.: 6.400   1st Qu.:0.2300   1st Qu.:0.2500   1st Qu.: 1.800  
 Median : 7.000   Median :0.2900   Median :0.3100   Median : 3.000  
 Mean   : 7.215   Mean   :0.3397   Mean   :0.3186   Mean   : 5.443  
 3rd Qu.: 7.700   3rd Qu.:0.4000   3rd Qu.:0.3900   3rd Qu.: 8.100  
 Max.   :15.900   Max.   :1.5800   Max.   :1.6600   Max.   :65.800  
   chlorides          density       Class   
 Min.   :0.00900   Min.   :0.9871   0:4898  
 1st Qu.:0.03800   1st Qu.:0.9923   1:1599  
 Median :0.04700   Median :0.9949           
 Mean   :0.05603   Mean   :0.9947           
 3rd Qu.:0.06500   3rd Qu.:0.9970           
 Max.   :0.61100   Max.   :1.0390           

# BUILD A NAIVE BAYES CLASSIFIER

In [39]:
install.packages("naivebayes")
library(naivebayes)
install.packages("pROC")
library(pROC)

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


# Make a function that quickly tests NBC accuracy

In [40]:
test_nbc_accuracy <- function(df, plot=FALSE) {
 
    # df = select(df, -Color)
    # split into test and training sets
    nrows <- nrow(df)
    smp_size <- floor(0.75 * nrows)

    set.seed(8720885)
    train_ind <- sample(seq_len(nrows), size = smp_size)
    
    train <- df[train_ind, ]
    test <- df[-train_ind, ]
    
    # create the Naive Bayes Classifer (nbc)
    nbc <- naive_bayes(Class ~ ., data = train)
    
    if (plot) {
        plot(nbc)
    }
    # Make a prediction & test accuracy
    test_pred <- predict(nbc, test,type = "prob")
    
    ### junky debug stuff
    # print(head (test_pred))
    ### junky debug stuff
    accuracy <- sum(test[,"Class"]==test_pred)/length(test_pred)
    print(paste("Accuracy=", round(accuracy*100,2), "%", sep=""))

    # try to do the AUC
    # print(paste("AUC=", auc(test_pred, df$Class)))

}

## Classification Accuracy, All Attributes

In [41]:
test_nbc_accuracy(all_data)
test_nbc_accuracy(all_data2)

[1] "Accuracy=0.62%"
[1] "Accuracy=0.62%"


## Classification Accuracy, Sensor-supported Attributes

In [42]:
test_nbc_accuracy(data)

[1] "Accuracy=0.43%"


## Comparing Area Under the (Receiver/Operator) Curve (AOC)

# OPTIMISING AUC

# OPTIMISING ATTRIBUTES

# ----------------------------------------------------------

In [44]:
    df = data
    plot = TRUE

    # split into test and training sets
    nrows <- nrow(df)
    smp_size <- floor(0.75 * nrows)

    set.seed(8720885)
    train_ind <- sample(seq_len(nrows), size = smp_size)
    
    train <- df[train_ind, ]
    test <- df[-train_ind, ]
    
    # create the Naive Bayes Classifer (nbc)
    nbc <- naive_bayes(Class ~ ., data = train)
    
    if (plot) {
        plot(nbc)
    }
    # Make a prediction & test accuracy
    test_pred <- predict(nbc, test,type = "prob")
    
    ### junky debug stuff
    # print(head (test_pred))
    ### junky debug stuff
    accuracy <- sum(test[,"Class"]==test_pred)/length(test_pred)
    print(paste("Accuracy=", round(accuracy*100,2), "%", sep=""))

    # try to do the AUC
    # print(paste("AUC=", auc(test_pred, df$Class)))
    # as.numeric(performance(ROCRpred, "auc")@y.values)


[1] "Accuracy=0.43%"


In [ ]:
Maybe try [this instead](http://blog.revolutionanalytics.com/2016/11/calculating-auc.html)

In [ ]:
Maybe try [this instead](https://www.r-bloggers.com/illustrated-guide-to-roc-and-auc/)

In [52]:
colnames(test_pred) <- c("White", "Red")
head(test_pred, 3)
head(test_pred[,'White'], 3)
head(test_pred[,'Red'], 3)


White,Red
6.104496e-05,0.9999390
1.136686e-03,0.9988633
8.027409e-04,0.9991973


[1] 6.104496e-05 1.136686e-03 8.027409e-04

[1] 0.9999390 0.9988633 0.9991973

In [53]:

whitevec <- ifelse(test_pred[,'White'] > test_pred[,'Red'] , 1, 0)
redvec   <- ifelse(test_pred[,'Red'] > test_pred[,'White'] , 1, 0)
head(whitevec, 3)
head(redvec, 3)



predvec <- ifelse(pred=="Lost", 1, 0)
realvec <- ifelse(realResults=="Lost", 1, 0)
pr <- prediction(predvec, realvec)


[1] 0 0 0

[1] 1 1 1

From [Cross Validated StackExchange](https://stats.stackexchange.com/) article on [R - Plotting a ROC curve for a Naive Bayes classifier using ROCR. Not sure if I'm plotting it correctly
](https://stats.stackexchange.com/questions/191805/r-plotting-a-roc-curve-for-a-naive-bayes-classifier-using-rocr-not-sure-if-i?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa)

In [ ]:
library (ROCR)

In [33]:
#BUILD THE NAIVE MODEL
# naive = naive.bayes(trainingdata, "Status")
#fitted = bn.fit(nbc, train, method="bayes", iss=1)

##RUN PREDICTION AND GET PROBABILITIES
# pred = predict(fitted, testdata, prob=TRUE)
results_prob = data.frame(t(attributes(test_pred)$prob))
head(test_pred)
#x <- cbind(pred, results_prob)
# head(x)

ERROR: Error in t.default(attributes(test_pred)$prob): argument is not a matrix


In [ ]:
Maybe try [this instead]()

## Run some Tests

## Run some Tests

## Run some Tests

## Assume the cheapest solution is a single sensor.  Try all the single-feature options.

In [ ]:
test_nbc_accuracy(select(data, fixed.acidity, Class))
test_nbc_accuracy(select(data, volatile.acidity, Class))
test_nbc_accuracy(select(data, citric.acid, Class))
test_nbc_accuracy(select(data, residual.sugar, Class))
test_nbc_accuracy(select(data, chlorides, Class))
test_nbc_accuracy(select(data, density, Class))

## Run some Tests

Yeah, that didn't work; but it was a quick test, and would have led to a cheaper product if it did.  Was still worth a shot.

## Do some plots

In [ ]:
test_nbc_accuracy(data, plot=TRUE)

In [ ]:
# Try Dropping chlorides, since it's the opposite of residual sugar

In [ ]:
test_data = select(data, -chlorides)
head(test_data)

In [ ]:
test_nbc_accuracy(test_data)

In [ ]:
# Try Dropping Volatile acidity

In [ ]:
test_data = select(test_data, -volatile.acidity)
head(test_data)

In [ ]:
test_nbc_accuracy(test_data)
test_nbc_accuracy(data)
test_nbc_accuracy(all_data)
test_nbc_accuracy(select(all_data, -quality))

In [ ]:


auc(test_data$, test_data$Class)

# ====================< JUNK >=====================

# Create a Naive Bayes Model, Try a Prediction

In [ ]:
nrows <- nrow(data)
smp_size <- floor(0.75 * nrows)

## set the seed to make your partition reproductible
set.seed(8720885)
train_ind <- sample(seq_len(nrows), size = smp_size)

In [ ]:
train <- data[train_ind, ]
test <- data[-train_ind, ]

In [ ]:
options(repr.plot.width=4, repr.plot.height=4)
plot(nbc)

In [ ]:
install.packages("naivebayes")
library(naivebayes)
nbc <- naive_bayes(Class ~ ., data = train)
plot(nbc)

In [ ]:
# Predict the test data
test_pred <- predict(nbc, test,type = "prob")

accuracy <- sum(test[,"Class"]==test_pred)/length(test_pred)
print(paste("Accuracy=", round(accuracy*100,2), "%", sep=""))

## Pull out a handy plotting function from last quarter

In [ ]:
require(ggplot2)
plot.density = function(df, col, xlabel, title) {
    ggplot(df, aes(col)) + 
          geom_bar() + 
          xlab(xlabel) + 
          ggtitle(title) + 
          theme(plot.title = element_text(hjust = 0.5, size=20, face="bold"),
               axis.title.x = element_text(size=15))
}

In [ ]:
options(repr.plot.width=12, repr.plot.height=4)
plot.density(data, data$Class, "1 = Red?, 2 = White?", "Wine Color")

## Try some Fancy Matrix Plotting

In [ ]:
require(GGally)

plot.matrix = function(df, cols){
    options(repr.plot.width=8, repr.plot.height=8)
    ggpairs(data=df,
            columns=cols,
            upper = list(continuous = "density"),
            lower = list(continuous = "density"),
            cardinality_threshold = 130)
}

In [ ]:
plot.matrix(data, c(1:6,7))

In [ ]:
prediction = predict(nbc, test, type= "prob")
head(prediction)